In [63]:
import requests
from requests.auth import HTTPBasicAuth
from flask import Flask, request
import threading
import webbrowser
import time
import pandas as pd

app = Flask(__name__)
authorization_code = None

@app.route('/callback')
def callback():
    global authorization_code
    authorization_code = request.args.get('code')
    return "Authorization code recebido. Você pode fechar esta janela."

def start_flask_app():
    app.run(port=5000)

def get_authorization_code(client_id, redirect_uri):
    threading.Thread(target=start_flask_app).start()
    time.sleep(1)
    
    state = "8bde85dd6e729dcd6e0d01dde003469d"
    url = f"https://www.bling.com.br/Api/v3/oauth/authorize?response_type=code&client_id={client_id}&redirect_uri={redirect_uri}&state={state}"
    webbrowser.open(url)

    # Aguarda até que o `authorization_code` seja definido
    while not authorization_code:
        time.sleep(1)
    
    return authorization_code

def get_access_token(client_id, client_secret, authorization_code):
    url = "https://www.bling.com.br/Api/v3/oauth/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "1.0"
    }
    body = {
        "grant_type": "authorization_code",
        "code": authorization_code,
        "redirect_uri": "http://localhost:5000/callback"
    }
    response = requests.post(url, headers=headers, data=body, auth=HTTPBasicAuth(client_id, client_secret))
    return response.json()

# Substitua com seus dados reais
CLIENT_ID = 'ef459eea1436d8e822687e2eca165773e94c95bb'
CLIENT_SECRET = '110185db75b656ffeca26f959c3a426f835ab5d704edf691c1cbd34ed465'
REDIRECT_URI = 'http://localhost:5000/callback'  # A URI de redirecionamento registrada no seu aplicativo Bling

# 1. Obtenha o authorization_code
authorization_code = get_authorization_code(CLIENT_ID, REDIRECT_URI)

# 2. Obtenha o access_token usando o authorization_code
token_response = get_access_token(CLIENT_ID, CLIENT_SECRET, authorization_code)
access_token = token_response.get('access_token')

print("Access Token:", access_token)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Access Token: 7b0914cbc7506d2dcfed6badf4da60c5bc42a3c0


In [64]:
url = "https://www.bling.com.br/Api/v3/produtos?pagina=1&limite=100&criterio=1&tipo=T"

headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

response = requests.get(url, headers=headers)
data = response.json()
#display(data)# Vamos assumir que a chave para acessar os produtos é 'data' na resposta JSON
produtos = data['data']

# Criando um DataFrame com os produtos
prod_df = pd.DataFrame(produtos)

In [52]:
prod_df

,id,nome,codigo,preco,tipo,situacao,formato,descricaoCurta,imagemURL
0,16256751090,Moletom Ellus Masculino Hoodie Basico Easa Azu...,299429,409.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",
1,16256751055,Moletom Ellus Masculino Hoodie Basico Easa Azu...,299428,409.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",
2,16256751020,Moletom Ellus Masculino Hoodie Basico Easa Azu...,299427,409.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",
3,16256750993,Moletom Ellus Masculino Hoodie Basico Easa Azu...,299426,409.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",
4,16256750984,Moletom Ellus Masculino Hoodie Basico Easa Azu...,299425,409.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",
...,...,...,...,...,...,...,...,...,...
95,16256075209,Blusa Calvin Klein Jeans Masculina Manga Longa...,299334,229.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",
96,16256075200,Blusa Calvin Klein Jeans Masculina Manga Longa...,299333,229.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",
97,16256075191,Blusa Calvin Klein Jeans Masculina Manga Longa...,299332,229.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",
98,16256074018,Blusa Calvin Klein Jeans Masculina Manga Longa...,299331,229.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",


In [45]:
def consultar_estoque():
    # Configuração da API
    base_url = "https://www.bling.com.br/Api/v3/estoques/saldos"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }

    # Solicitação de dados do usuário
    id_produto = input("Digite o ID do produto: ")
    codigo_produto = input("Digite o código do produto: ")

    # Montagem da URL com parâmetros
    url = f"{base_url}?idsProdutos[]={id_produto}&codigo={codigo_produto}"

    # Fazendo a requisição à API
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()

        # Extraindo os dados de saldo
        saldo_info = data.get('data', [])
        if saldo_info:
            produto_info = saldo_info[0].get('produto', {})
            saldo_fisico_total = saldo_info[0].get('saldoFisicoTotal', 'N/A')
            saldo_virtual_total = saldo_info[0].get('saldoVirtualTotal', 'N/A')
            depositos_info = saldo_info[0].get('depositos', [])

            # Criando um DataFrame com as informações do saldo
            df = pd.DataFrame(depositos_info)
            df['produto_id'] = produto_info.get('id', 'N/A')
            df['produto_codigo'] = codigo_produto
            df['saldo_fisico_total'] = saldo_fisico_total
            df['saldo_virtual_total'] = saldo_virtual_total
            df.rename(columns={
                'id': 'deposito_id',
                'saldoFisico': 'saldo_fisico',
                'saldoVirtual': 'saldo_virtual'
            }, inplace=True)

            display(df)
        else:
            print("Nenhum saldo encontrado para o produto.")
    else:
        print("Erro ao acessar a API:", response.status_code, response.text)

# Chamando a função para testar
consultar_estoque()


,deposito_id,saldo_fisico,saldo_virtual,produto_id,produto_codigo,saldo_fisico_total,saldo_virtual_total
0,863558208,7,7,16205272848,291414,7,7
1,6938657703,0,0,16205272848,291414,7,7


In [46]:
url = "https://www.bling.com.br/Api/v3/depositos?pagina=1&limite=100&situacao=1"

headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

response = requests.get(url, headers=headers)
data = response.json()
#display(data)# Vamos assumir que a chave para acessar os produtos é 'data' na resposta JSON
depositos = data['data']

# Criando um DataFrame com os produtos
df = pd.DataFrame(depositos)
display(df)

,id,descricao,situacao,padrao,desconsiderarSaldo
0,863558208,Escritório,1,True,False
1,6938657703,Loja física,1,False,False


In [65]:
import requests
import pandas as pd


# ID do depósito "Escritório"
ID_DEPOSITO_ESCRITORIO = 863558208

# Função para consultar o saldo de um produto específico
def consultar_estoque(id_produto, codigo_produto):
    base_url = "https://www.bling.com.br/Api/v3/estoques/saldos"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }

    # Montagem da URL com parâmetros
    url = f"{base_url}?idsProdutos[]={id_produto}&codigo={codigo_produto}"

    # Fazendo a requisição à API
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()

        # Extraindo os dados de saldo
        saldo_info = data.get('data', [])
        if saldo_info:
            produto_info = saldo_info[0].get('produto', {})
            saldo_fisico_total = saldo_info[0].get('saldoFisicoTotal', 'N/A')
            saldo_virtual_total = saldo_info[0].get('saldoVirtualTotal', 'N/A')
            depositos_info = saldo_info[0].get('depositos', [])

            # Filtrar saldo do depósito "Escritório"
            saldo_escritorio = next((d['saldoVirtual'] for d in depositos_info if d['id'] == ID_DEPOSITO_ESCRITORIO), 'N/A')

            return {
                'produto_id': produto_info.get('id', 'N/A'),
                'produto_codigo': codigo_produto,
                'saldo_fisico_total': saldo_fisico_total,
                'saldo_virtual_total': saldo_virtual_total,
                'saldo_escritorio': saldo_escritorio
            }
        else:
            return {
                'produto_id': id_produto,
                'produto_codigo': codigo_produto,
                'saldo_escritorio': 'N/A'
            }
    else:
        print(f"Erro ao acessar a API para produto {id_produto}: {response.status_code}, {response.text}")
        return {
            'produto_id': id_produto,
            'produto_codigo': codigo_produto,
            'saldo_escritorio': 'Erro'
        }

# Função para adicionar a coluna de estoque ao DataFrame
def obter_estoque_produtos(prod_df):
    # Função interna para buscar o estoque de cada produto
    def buscar_estoque(row):
        saldo_info = consultar_estoque(row['id'], row['codigo'])
        try:
            saldo_escritorio = int(saldo_info['saldo_escritorio'])
        except ValueError:
            saldo_escritorio = 0
        return saldo_escritorio

    # Adicionar a coluna "estoque" ao DataFrame
    prod_df['estoque'] = prod_df.apply(buscar_estoque, axis=1)
    return prod_df

def obter_todos_os_produtos():
    base_url = "https://www.bling.com.br/Api/v3/produtos"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }

    pagina = 1
    limite = 100  # Limite por página (ajuste se necessário)
    prod_df = pd.DataFrame()

    while True:
        url = f"{base_url}?pagina={pagina}&limite={limite}&criterio=1&tipo=T"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            produtos_data = data.get('data', [])

            # Concatenar os dados ao DataFrame
            prod_df = pd.concat([prod_df, pd.DataFrame(produtos_data)], ignore_index=True)

            # Verificar se há mais páginas
            if len(produtos_data) < limite:
                break  # Última página alcançada
            else:
                pagina += 1
        else:
            print(f"Erro ao acessar a API na página {pagina}: {response.status_code}, {response.text}")
            break

    return prod_df

# Obter todos os produtos
prod_df = obter_todos_os_produtos()

# Adicionar a coluna "estoque" ao DataFrame
prod_df = obter_estoque_produtos(prod_df)

# Filtrar apenas produtos com estoque superior a zero
prod_df_filtrado = prod_df[prod_df['estoque'] > 0]

# Mostrar o DataFrame filtrado
display(prod_df_filtrado)

Erro ao acessar a API na página 595: 504, <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>www.bling.com.br | 504: Gateway time-out</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 font-light text-6

KeyboardInterrupt: 

In [71]:
import requests
from requests.auth import HTTPBasicAuth
from flask import Flask, request
import threading
import webbrowser
import time
import pandas as pd

app = Flask(__name__)
authorization_code = None

@app.route('/callback')
def callback():
    global authorization_code
    authorization_code = request.args.get('code')
    return "Authorization code recebido. Você pode fechar esta janela."

def start_flask_app():
    app.run(port=5000)

def get_authorization_code(client_id, redirect_uri):
    threading.Thread(target=start_flask_app).start()
    time.sleep(1)
    
    state = "8bde85dd6e729dcd6e0d01dde003469d"
    url = f"https://www.bling.com.br/Api/v3/oauth/authorize?response_type=code&client_id={client_id}&redirect_uri={redirect_uri}&state={state}"
    webbrowser.open(url)

    # Aguarda até que o `authorization_code` seja definido
    while not authorization_code:
        time.sleep(1)
    
    return authorization_code

def get_access_token(client_id, client_secret, authorization_code):
    url = "https://www.bling.com.br/Api/v3/oauth/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "1.0"
    }
    body = {
        "grant_type": "authorization_code",
        "code": authorization_code,
        "redirect_uri": "http://localhost:5000/callback"
    }
    response = requests.post(url, headers=headers, data=body, auth=HTTPBasicAuth(client_id, client_secret))
    return response.json()

def consultar_estoque(id_produto, codigo_produto):
    base_url = "https://www.bling.com.br/Api/v3/estoques/saldos"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }

    # Montagem da URL com parâmetros
    url = f"{base_url}?idsProdutos[]={id_produto}&codigo={codigo_produto}"

    # Fazendo a requisição à API
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()

        # Extraindo os dados de saldo
        saldo_info = data.get('data', [])
        if saldo_info:
            produto_info = saldo_info[0].get('produto', {})
            saldo_fisico_total = saldo_info[0].get('saldoFisicoTotal', 'N/A')
            saldo_virtual_total = saldo_info[0].get('saldoVirtualTotal', 'N/A')
            depositos_info = saldo_info[0].get('depositos', [])

            # Filtrar saldo do depósito "Escritório"
            saldo_escritorio = next((d['saldoVirtual'] for d in depositos_info if d['id'] == ID_DEPOSITO_ESCRITORIO), 'N/A')

            return {
                'produto_id': produto_info.get('id', 'N/A'),
                'produto_codigo': codigo_produto,
                'saldo_fisico_total': saldo_fisico_total,
                'saldo_virtual_total': saldo_virtual_total,
                'saldo_escritorio': saldo_escritorio
            }
        else:
            return {
                'produto_id': id_produto,
                'produto_codigo': codigo_produto,
                'saldo_escritorio': 'N/A'
            }
    else:
        print(f"Erro ao acessar a API para produto {id_produto}: {response.status_code}, {response.text}")
        return {
            'produto_id': id_produto,
            'produto_codigo': codigo_produto,
            'saldo_escritorio': 'Erro'
        }
    
def obter_estoque_produtos(prod_df):
    # Função interna para buscar o estoque de cada produto
    def buscar_estoque(row):
        saldo_info = consultar_estoque(row['id'], row['codigo'])
        try:
            saldo_escritorio = int(saldo_info['saldo_escritorio'])
        except ValueError:
            saldo_escritorio = 0
        return saldo_escritorio

    # Adicionar a coluna "estoque" ao DataFrame
    prod_df['estoque'] = prod_df.apply(buscar_estoque, axis=1)
    return prod_df

def obter_todos_os_produtos():
    base_url = "https://www.bling.com.br/Api/v3/produtos"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }

    pagina = 1
    limite = 450  # Limite por página (ajuste se necessário)
    prod_df = pd.DataFrame()

    while True:
        url = f"{base_url}?pagina={pagina}&limite={limite}&criterio=1&tipo=T"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            produtos_data = data.get('data', [])

            # Concatenar os dados ao DataFrame
            prod_df = pd.concat([prod_df, pd.DataFrame(produtos_data)], ignore_index=True)

            # Verificar se há mais páginas
            if len(produtos_data) < limite:
                break  # Última página alcançada
            else:
                pagina += 1
        else:
            print(f"Erro ao acessar a API na página {pagina}: {response.status_code}, {response.text}")
            break

    return prod_df


# Substitua com seus dados reais
CLIENT_ID = 'ef459eea1436d8e822687e2eca165773e94c95bb'
CLIENT_SECRET = '110185db75b656ffeca26f959c3a426f835ab5d704edf691c1cbd34ed465'
REDIRECT_URI = 'http://localhost:5000/callback'  # A URI de redirecionamento registrada no seu aplicativo Bling

# 1. Obtenha o authorization_code
authorization_code = get_authorization_code(CLIENT_ID, REDIRECT_URI)

# 2. Obtenha o access_token usando o authorization_code
token_response = get_access_token(CLIENT_ID, CLIENT_SECRET, authorization_code)
access_token = token_response.get('access_token')

url = "https://www.bling.com.br/Api/v3/produtos?pagina=1&limite=100&criterio=1&tipo=T"

headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

response = requests.get(url, headers=headers)
data = response.json()
#display(data)# Vamos assumir que a chave para acessar os produtos é 'data' na resposta JSON
produtos = data['data']

# Criando um DataFrame com os produtos
prod_df = pd.DataFrame(produtos)

# ID do depósito "Escritório"
ID_DEPOSITO_ESCRITORIO = 863558208

# Obter todos os produtos
prod_df = obter_todos_os_produtos()

# Adicionar a coluna "estoque" ao DataFrame
prod_df = obter_estoque_produtos(prod_df)

# Filtrar apenas produtos com estoque superior a zero
prod_df_filtrado = prod_df[prod_df['estoque'] > 0]

# Salvar o DataFrame filtrado em um arquivo CSV
prod_df_filtrado.to_csv("produtos_com_estoque.csv", index=False)

print("Dados salvos com sucesso no arquivo 'produtos_com_estoque.csv'")


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
